# Exploring a rose garden with SPARQL

*Philippe Rocca-Serra (philippe.rocca-serra[at]oerc.ox.ac.uk), University of Oxford e-Research Centre

In [1]:
from rdflib import Graph, RDF
from IPython.core.display import display, HTML
import os
import json
import csv
import uuid

In [2]:
def queryResultToHTMLTable(queryResult):
   HTMLResult = '<table><tr style="color:white;background-color:#43BFC7;font-weight:bold">'
   # print variable names
   for varName in queryResult.vars:
       HTMLResult = HTMLResult + '<td>' + varName + '</td>'
   HTMLResult = HTMLResult + '</tr>'
   # print values from each row
   for row in queryResult:
      HTMLResult = HTMLResult + '<tr>'   
      for column in row:
         HTMLResult = HTMLResult + '<td>' + column + '</td>'
      HTMLResult = HTMLResult + '</tr>'
   HTMLResult = HTMLResult + '</table>'
   display(HTML(HTMLResult))

*credits to Bob du Charme for the following function [http://www.snee.com/bobdc.blog/2016/07/sparql-in-a-jupyter-aka-ipytho.html]

In [3]:
g = Graph()

Let's read the RDF graph generated using the rose-dtpkg2rdf.py python script and saved to disk as a turtle file

In [4]:
g.parse("./rose-data/rose-aroma-test.ttl", format="n3")

<Graph identifier=Ne40ee8a3e63c4712892b7246ad23c0c0 (<class 'rdflib.graph.Graph'>)>

Now let's ask for the independent variables and their levels using the following SPARQL query

In [5]:
qres1 = g.query("""
PREFIX stato: <http://purl.obolibrary.org/obo/STATO_>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX ncbitax: <http://purl.obolibrary.org/obo/NCBITax_>
prefix ro: <http://purl.obolibrary.org/obo/RO_> 
            SELECT DISTINCT
             ?Predictor
             ?PredictorLevel
             WHERE { 
                ?var a stato:0000087 ;
                    rdfs:label ?Predictor;
                    ro:has_part ?value.
                ?value rdfs:label ?PredictorLevel    
                 }               
""")

We can display the results of that query using the function declared earlier on.

In [6]:
queryResultToHTMLTable(qres1)

Predictor,PredictorLevel
genotype,R. chinensis 'Old Blush'
genotype,R. gigantea
organism part,sepal
organism part,stamen
organism part,petal


Let's now ask for the number of biological and technical replicates used to compute the mean concentration of the chemical compounds detected and forming the signature of the rose fragrance.

In [7]:
qres8 = g.query("""
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix chmo:   <http://purl.obolibrary.org/obo/CHMO_> 
prefix msio:   <http://purl.obolibrary.org/obo/MSIO_> 
prefix stato: <http://purl.obolibrary.org/obo/STATO_> 
prefix obi: <http://purl.obolibrary.org/obo/OBI_> 
prefix ro: <http://purl.obolibrary.org/obo/RO_>
prefix po: <http://purl.obolibrary.org/obo/PO_>

SELECT 
        
        ?TreatmentGroup 
        ?MeanConcentration
        ?ChemicalCompound        
        (count(distinct ?member) as ?NbTechnicalReplicate) 
        (count(distinct ?input) as ?NbBiologicalReplicate) 
      WHERE {
            ?population a stato:0000193 ;
                rdfs:label ?TreatmentGroup ;
                ro:has_member ?member .      
            ?member ro:has_specified_input ?input .              
            ?mean a stato:0000402 ;
                stato:computed_over ?population ;
                ro:has_value ?MeanConcentration ;
                ro:is_about ?ChemicalCompound .
            ?concentration a stato:0000072;
                ro:is_specified_output_of ?assay ;
                ro:is_about ?ChemicalCompound .      

            }
      GROUP BY ?population 
""")


Once more, we invoked the pretty printing function:

In [8]:
queryResultToHTMLTable(qres8)

TreatmentGroup,MeanConcentration,ChemicalCompound,NbTechnicalReplicate,NbBiologicalReplicate
R. chinensis 'Old Blush' sepals,NA,chebi:88370,3,1
R. chinensis 'Old Blush' stamens,NA,chebi:88370,3,1
R. chinensis 'Old Blush' petals,NA,chebi:88370,3,1
R. gigantea petals,NA,chebi:88370,3,1
